In [53]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
from crewai import Crew, Agent, Task


In [55]:
import os

def get_openai_api_key ():
    return os.getenv("OPENAI_API_KEY")

from dotenv import load_dotenv
load_dotenv()

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-4"

In [56]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal = "Identify high-value leads that match "
         "our ideal customer profile",
    backstory=  "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth.",
    allow_delegation= True,
    verbose= False

)

In [57]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [58]:
from crewai_tools import SerperDevTool,\
                         FileReadTool,\
                          DirectoryReadTool

In [59]:
file_read_tool = FileReadTool()
search_tool = SerperDevTool()
directory_tool = DirectoryReadTool( directory='./instructions')

In [60]:
reach_out = Task (
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
),
    expected_output=(      
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
),
    tools =[directory_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,

)

In [61]:
from crewai.tools import BaseTool

In [62]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [63]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [64]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [65]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [67]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose= False,
	memory=True
)

In [68]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Lead Sales Representative
## Task: Search the internet for DeepLearningAI company profile


# Agent: Lead Sales Representative
## Final Answer: 
DeepLearningAI is a well-respected education technology company dedicated to providing comprehensive Artificial Intelligence education to the global community. As online learning platforms continue to surge, it's crucial we understand companies like DeepLearningAI.

1. Background: DeepLearningAI is known for offering courses that help students gain the AI skills needed in today's rapidly advancing digital world. The company was founded by AI pioneer and Stanford Professor Andrew Ng, who is largely credited for his work at Google Brain and co-founding Coursera. DeepLearning.AI courses cut across Deep Learning Specialization, AI for Everyone, AI for Medicine, and TensorFlow Developer Certificate, among others. 

2. Key Decision Makers: The key decision-maker at DeepLearningAI is its founder Andrew Ng. Having multiple roles as a professo

In [70]:
from IPython.display import Markdown
Markdown(result.raw)

Here are the final personalized email drafts to be sent to Andrew Ng:

1. Dear Andrew Ng,

I hope this message finds you well.

Having closely followed DeepLearningAI's inspiring journey in expanding AI education, I'm reaching out with some thoughts on how we can collaborate and propel this mission forward.

Your recent launch of the AI Engineering Professional Certificate is a commendable step towards practical AI training. By integrating our AI-powered tools, we can offer interactive, hands-on learning experiences that could be a game-changer in the e-learning space.

Looking forward to hearing your thoughts.

Sincerely,
[Your Name]
  
2. Dear Andrew Ng,

Just like DeepLearningAI, we also believe in the power of data. Understanding and acting on learner behavior can greatly enhance the effectiveness of your courses. 

Our data analytics toolkit can provide you deep insights into patterns and trends amongst learners. Armed with this data, DeepLearningAI can further tailor the learning experience to meet student needs, thus creating more successful AI professionals.

Looking forward to helping you unlock the full potential of your learners.

Cheers,
[Your Name] 

3. Dear Andrew Ng,

In the edtech space, staying ahead of the curve is paramount for success. DeepLearningAI's commitment to providing cutting-edge learning opportunities is commendable.

To support this vision, I'd like to introduce our customizable AI solutions. These adaptable tools can be seamlessly integrated into your course development process to create unique, personalized learning pathways for students—an invaluable feature in today's diverse learning landscape.

Eager to hear your thoughts on integrating our AI technology into DeepLearningAI's future.

Best Regards,
[Your Name]